# Our data tends to look way worse... let's import a dataset with "noise"

In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")  # This is to ignore any warnings that might pop up during execution

In [2]:
# Basic libraries to manipulate data
import matplotlib.pyplot as plt  # Matplotlib for data visualization
import numpy as np  # Numpy for numerical computations
import pandas as pd  # Pandas for data manipulation

In [3]:
np.random.seed(42)  # To ensure all the probabilistic things are reproducible

In [4]:
# Specify the path to the datasets
data_path = "./data/"

# Specify the filenames of the datasets
survey_filename = "survey_music_noise.csv"

# Read the CSV files and create backup copies
survey_df_data = pd.read_csv(data_path + survey_filename)

# Create working copies of the dataframes for analysis
survey_df = survey_df_data.copy()

Last time we saw how useful it was to get the correct dtype in a column because of the different functions are way more efficient and easy to define. 
In this notebook we will see all the different values of the columns that don't allow us to get a correct prediction, correct them and convert each column to the correct dtype. Then we will diagnose the problems with our dataset (a step called **Data Profiling**) in order to get a clean dataset in the next notebook

In [5]:
# Let's check its structure
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779 entries, 0 to 778
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Timestamp                     766 non-null    object
 1   Age                           759 non-null    object
 2   Primary streaming service     767 non-null    object
 3   Hours per day                 765 non-null    object
 4   While working                 769 non-null    object
 5   Instrumentalist               761 non-null    object
 6   Composer                      766 non-null    object
 7   Fav genre                     765 non-null    object
 8   Exploratory                   769 non-null    object
 9   Foreign languages             755 non-null    object
 10  BPM                           660 non-null    object
 11  Frequency [Classical]         767 non-null    object
 12  Frequency [Country]           764 non-null    object
 13  Frequency [EDM]     

# Timestamp

Since all of the columns appear as object we can access their str methods. This will help us see if the values are of the length we require

In [6]:
# Let's check the count for each component (including NA values)
survey_df['Timestamp'].str.len().value_counts(dropna=False) 

18.0    417
17.0    284
16.0     49
NaN      13
19.0     10
1.0       3
4.0       3
Name: Timestamp, dtype: int64

The records that have length less than 16 cannot be a true date. We will inspect these rows first obtaining a mask over the ones with length fewer than 16

In [7]:
less_than_16=survey_df['Timestamp'][survey_df['Timestamp'].
                                    str.len()<16]
less_than_16

79        0
234    -500
253       0
345    -500
603    Null
765       0
Name: Timestamp, dtype: object

These are clearly null values so we substitute them into a None value which is more suited to denote a missing value in an object dataframe

In [8]:
survey_df['Timestamp'][survey_df['Timestamp'].str.len()<16]=np.nan

In [9]:
survey_df['Timestamp'].str.len().value_counts(dropna=False) 

18.0    417
17.0    284
16.0     49
NaN      19
19.0     10
Name: Timestamp, dtype: int64

We'll try to convert it as it is now into a datetime dtype

In [10]:
#pd.to_datetime(survey_df['Timestamp'])

In [11]:
survey_df['Timestamp'].unique()

array([nan, '9/28/2022 17:25:48', '9/13/2022 16:16:16',
       '9/10/2022 9:06:07', '8/29/2022 2:46:38', '8/28/2022 14:12:55',
       '9/3/2022 17:14:57', '8/28/2022 11:08:51', '9/12/2022 18:10:24',
       '8/29/2022 0:03:43', '8/29/2022 4:37:05', '8/28/2022 17:33:12',
       '10/5/2022 12:30:23', '8/29/2022 10:44:43', '9/21/2022 21:12:20',
       '8/28/2022 19:16:43', '8/28/2022 21:48:29', '9/1/2022 16:58:12',
       '8/29/2022 18:22:58', '8/28/2022 12:54:35', '10/6/2022 3:35:11',
       '9/15/2022 15:30:41', '8/29/2022 9:42:23', '8/29/2022 2:43:40',
       '9/1/2022 15:21:55', '8/29/2022 1:34:27', '8/29/2022 6:58:24',
       '8/28/2022 21:14:34', '8/28/2022 11:25:49', '10/4/2022 8:53:39',
       '9/2/2022 14:09:04', '8/31/2022 9:42:46', '8/29/2022 19:07:50',
       '8/31/2022 8:35:00', '8/28/2022 18:55:44', '9/1/2022 17:00:07',
       '8/28/2022 18:35:17', '8/28/2022 15:22:00', '8/30/2022 16:04:47',
       '8/29/2022 4:10:25', '8/29/2022 5:08:52', '8/28/2022 12:39:49',
       '10/4/2

The error is informative since it shows we have data of the type "8-30-2022 0.14.35" while al the data we had seen had a format of days separated from months and years with a / and hour, minute and second separated with :

We will replace each of these symbols using the replace method of the str accessor we've previously reviewed. This function takes a pattern to replace and then what its going to be replaced by

In [12]:
survey_df['Timestamp']=survey_df['Timestamp'].str.replace('.',':')
survey_df['Timestamp']=survey_df['Timestamp'].str.replace('-','/')

With these substitutions we can finally convert it into a datetime dtype

In [13]:
survey_df['Timestamp']=pd.to_datetime(survey_df['Timestamp'])
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779 entries, 0 to 778
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Timestamp                     760 non-null    datetime64[ns]
 1   Age                           759 non-null    object        
 2   Primary streaming service     767 non-null    object        
 3   Hours per day                 765 non-null    object        
 4   While working                 769 non-null    object        
 5   Instrumentalist               761 non-null    object        
 6   Composer                      766 non-null    object        
 7   Fav genre                     765 non-null    object        
 8   Exploratory                   769 non-null    object        
 9   Foreign languages             755 non-null    object        
 10  BPM                           660 non-null    object        
 11  Frequency [Classical]         76

With these dtype we can make operations more suited to datetime like obtaining the month out of the date. To use these methods we have to use the `.dt` accessor, similar to the `.str` accesor we used before

In [14]:
survey_df['Timestamp'].dt.month

0      NaN
1      9.0
2      9.0
3      9.0
4      8.0
      ... 
774    8.0
775    8.0
776    9.0
777    8.0
778    9.0
Name: Timestamp, Length: 779, dtype: float64

We have to do this for each column. And we can follow a similar strategy to the one above.

To see an output of all of the values that cannot be converted into a certain dtype we will need to create our own function.

# Numeric columns

## Age

Let's define a custom function that helps us see which values cannot be converted into floats

In [15]:
def find_non_convertible_floats(series):

    non_convertibles = set()
    for item in series:
        try:
            float(item)  # Try converting to float
        except ValueError:
            non_convertibles.add(item)  # Add to set if conversion fails
    return non_convertibles

find_non_convertible_floats(survey_df['Age'])

{'Null', 'fifteen', 'twenty'}

Now we see that the 'Null' value should be converted into a np.nan, and both the values fifteen and twenty should be replace with their numerical values. We can accomplish this with a replace dictionary

In [16]:
replace_dict_numeric={'Null':np.nan,'fifteen':15,'twenty':20}

survey_df['Age']=survey_df['Age'].replace(replace_dict_numeric)

# Now we can convert it to the correct dtype
survey_df['Age']=survey_df['Age'].astype(float)


## BPM

In [17]:
find_non_convertible_floats(survey_df['BPM'])

{'Null'}

In [18]:
survey_df['BPM']=survey_df['BPM'].replace(replace_dict_numeric)

# Now we can convert it to the correct dtype
survey_df['BPM']=survey_df['BPM'].astype(float)

## Hours per day

In [19]:
find_non_convertible_floats(survey_df['Hours per day'])

{'Null'}

In [20]:
survey_df['Hours per day']=survey_df['Hours per day'].replace(replace_dict_numeric)

# Now we can convert it to the correct dtype
survey_df['Hours per day']=survey_df['Hours per day'].astype(float)

## Mental health

In [21]:
for col in ['Anxiety','Depression','Insomnia','OCD']:
    print(find_non_convertible_floats(survey_df[col]))

{'Null'}
{'Null'}
{'Null'}
{'Null'}


In [22]:
for col in ['Anxiety','Depression','Insomnia','OCD']:
    survey_df[col]=survey_df[col].replace(replace_dict_numeric)

    # Now we can convert it to the correct dtype
    survey_df[col]=survey_df[col].astype(float)

In [23]:
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779 entries, 0 to 778
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Timestamp                     760 non-null    datetime64[ns]
 1   Age                           758 non-null    float64       
 2   Primary streaming service     767 non-null    object        
 3   Hours per day                 763 non-null    float64       
 4   While working                 769 non-null    object        
 5   Instrumentalist               761 non-null    object        
 6   Composer                      766 non-null    object        
 7   Fav genre                     765 non-null    object        
 8   Exploratory                   769 non-null    object        
 9   Foreign languages             755 non-null    object        
 10  BPM                           659 non-null    float64       
 11  Frequency [Classical]         76

# Categorical columns

We can see there are some columns that should be categorical, i.e. they can only take one value of a certain set of fixed values and they don't have any order between them (one isn't better than any other).

To see this, the best way is to see a value counts on each categorical column

In [24]:
survey_df['Primary streaming service'].value_counts(dropna=False)

Spotify                              471
YouTube Music                         96
I do not use a streaming service.     72
Apple Music                           51
Other streaming service               50
NaN                                   12
Pandora                               11
0                                      6
-500                                   2
Null                                   2
spotify                                1
YOUTUBE MUSIC                          1
other streaming service                1
SPOTIFY                                1
OTHER STREAMING SERVICE                1
 Spotify                               1
Name: Primary streaming service, dtype: int64

As we can see we have some values that are the same except they're capitalized differently. For example we have `Spotify, spotify, SPOTIFY`. 

And we also have some null values that take on the same kind of placeholders as before: `-500,Null,0`. 

The second problem we already know how to fix: With a replace dictionary. For the first one we will put all the strings into lowercase and remove any spaces at the end of the word with the function `strip` 